# IMPORT DATA DAN LIBRARY

In [100]:
import numpy as np
import pandas as pd

# split & CV
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# base & transformers
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.combine import SMOTETomek

# model & metrics 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [10]:
df = pd.read_csv('DataFrame_processed/DataFrame_processed.csv')

In [141]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'EducationField', 'EnvironmentSatisfaction',
       'HourlyRate', 'JobInvolvement', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'OverTime',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsInCurrentRole', 'ExperienceRatio',
       'IncomePerYearExp', 'TenureSatisfaction'],
      dtype='object')

In [13]:
X = df.drop(columns=["Attrition"])
y = df["Attrition"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# PIPELINE FULL

In [45]:
# pipeline transformasi
prep = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), selector(dtype_include=np.number)),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), selector(dtype_exclude=np.number)),
    ],
    remainder="drop"
)

# full preprocessing pipeline
pipe = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTETomek(random_state=42)),
    ("clf", RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1)) # bisa tambah ata ganti model lain 
])

> ## Untuk output score

In [218]:
from sklearn.metrics import make_scorer, fbeta_score

f2_scorer = make_scorer(fbeta_score, beta=2, average="binary")


In [47]:
# Silakan di copas 
pipe.fit(X_train, y_train)
y_pred  = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8856    0.9717    0.9266       247
           1     0.6957    0.3404    0.4571        47

    accuracy                         0.8707       294
   macro avg     0.7906    0.6560    0.6919       294
weighted avg     0.8552    0.8707    0.8516       294

Test ROC-AUC: 0.8154


---

---

# Model 1 - Logistic Regression

In [235]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42, max_iter=1000)

pipe_log_reg = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTETomek(random_state=42)),
    ("clf", logreg)
])

In [237]:
pipe_log_reg.fit(X_train, y_train)
y_pred  = pipe_log_reg.predict(X_test)
y_proba = pipe_log_reg.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.9317    0.7733    0.8451       247
           1     0.3708    0.7021    0.4853        47

    accuracy                         0.7619       294
   macro avg     0.6512    0.7377    0.6652       294
weighted avg     0.8420    0.7619    0.7876       294

Test ROC-AUC: 0.7941


> ## HyperParameter Tuning

In [245]:
# Hyperparameter ranges
C_range = np.logspace(-4, 2, 20)  
penalty_options = ['l1', 'l2', 'elasticnet']
solver_options = ['liblinear', 'saga', 'lbfgs']
l1_ratio_range = np.linspace(0, 1, 5)  

# hyperparameters_logreg = {
#     'clf__C': C_range,
#     'clf__penalty': penalty_options,
#     'clf__solver': solver_options,
#     'clf__class_weight': [None, 'balanced'],
#     'clf__l1_ratio': l1_ratio_range
# }

param_distributions_logreg = [
    # L2 penalty
    {
        'clf__penalty': ['l2'],
        'clf__solver': ['lbfgs', 'saga'],
        'clf__C': np.logspace(-3, 2, 10),
        'clf__max_iter': [500, 1000, 2000],
        'clf__class_weight': [None, 'balanced']
    },
    # L1 penalty
    {
        'clf__penalty': ['l1'],
        'clf__solver': ['liblinear', 'saga'],
        'clf__C': np.logspace(-3, 2, 10),
        'clf__max_iter': [500, 1000, 2000],
        'clf__class_weight': [None, 'balanced']
    },
    # ElasticNet penalty
    {
        'clf__penalty': ['elasticnet'],
        'clf__solver': ['saga'],
        'clf__C': np.logspace(-3, 2, 10),
        'clf__l1_ratio': np.linspace(0.1, 0.9, 5),
        'clf__max_iter': [500, 1000, 2000],
        'clf__class_weight': [None, 'balanced']
    }
]

rs_logreg = RandomizedSearchCV(pipe_log_reg, param_distributions=param_distributions_logreg , scoring=f2_scorer, random_state=42, cv=5, n_iter=50)
rs_logreg.fit(X_train, y_train)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEF787380>),
                                                                              ('cat',
                                                                               OneHotEncoder(handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEFE1FBF0>...
       1.66810054e-01, 5.99484250e-01, 2.15443469e+00, 7.74263683e+00,
       2.78255940e+01, 1.00000000e+02]),
                                         'clf__class_weight': [None,
                                                               'balanced'],
                                         'clf__l1_ratio': array([0.1, 0.3, 0.5, 0.7, 0.9]),
                                         'clf__max_iter': [500, 1000, 2000],
                                         'clf__penalty': ['elasticnet'],
                                         'clf__solver': ['saga']}],
                   random_state=42,
                   scoring=make_scorer(fbeta_score, response_method='predict', beta=2, average=binary))

In [303]:
print(f'score F2: {rs_logreg.best_score_}')
print(f'best param : {rs_logreg.best_params_}')

score F2: 0.6018068817726352
best param : {'clf__solver': 'saga', 'clf__penalty': 'l2', 'clf__max_iter': 500, 'clf__class_weight': 'balanced', 'clf__C': 0.1668100537200059}


In [309]:
logreg_tuned = rs_logreg.best_estimator_
logreg_tuned.fit(X_train, y_train)
y_pred  = logreg_tuned.predict(X_test)
y_proba = logreg_tuned.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))
print("Test F2:", rs_logreg.best_score_.round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.9363    0.7733    0.8470       247
           1     0.3778    0.7234    0.4964        47

    accuracy                         0.7653       294
   macro avg     0.6570    0.7483    0.6717       294
weighted avg     0.8470    0.7653    0.7909       294

Test ROC-AUC: 0.8002
Test F2: 0.6018


# Model 2 - Decision Tree Classifier

In [66]:
from sklearn.tree import DecisionTreeClassifier

dec_tree_clf = DecisionTreeClassifier(random_state=42)

pipe_tree = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTETomek(random_state=42)),
    ("clf", dec_tree_clf)
])

In [76]:
pipe_tree.fit(X_train, y_train)
y_pred  = pipe_tree.predict(X_test)
y_proba = pipe_tree.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8577    0.8300    0.8436       247
           1     0.2364    0.2766    0.2549        47

    accuracy                         0.7415       294
   macro avg     0.5471    0.5533    0.5493       294
weighted avg     0.7584    0.7415    0.7495       294

Test ROC-AUC: 0.5533


> ## HyperParameter Tuning

In [257]:
max_depth_tree = [int(x) for x in np.linspace(5, 50, 20)]  
split_tree = [int(x) for x in np.linspace(2, 50, 20)]      
leaf_tree = [int(x) for x in np.linspace(1, 50, 20)]       
max_features_tree = [None, 'sqrt', 'log2']                 
criterion_tree = ["gini", "entropy"]

hyperparameters_tree = {
    'clf__max_depth': max_depth_tree,
    'clf__min_samples_split': split_tree,
    'clf__min_samples_leaf': leaf_tree,
    'clf__max_features': max_features_tree,
    'clf__criterion': criterion_tree,
    'clf__class_weight': [None, "balanced"]
}

rs_tree = RandomizedSearchCV(pipe_tree, hyperparameters_tree, scoring=f2_scorer, random_state=42, cv=5, n_iter=50)
rs_tree.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEF787380>),
                                                                              ('cat',
                                                                               OneHotEncoder(handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEFE1FBF0>...
                                        'clf__max_depth': [5, 7, 9, 12, 14, 16,
                                                           19, 21, 23, 26, 28,
                                                           31, 33, 35, 38, 40,
                                                           42, 45, 47, 50],
                                        'clf__max_features': [None, 'sqrt',
                                                              'log2'],
                                        'clf__min_samples_leaf': [1, 3, 6, 8,
                                                                  11, 13, 16,
                                                                  19, 21, 24,
                                                                  26, 29, 31,
                                                                  34, 37, 39,
                                                                  42, 44, 47,
                                                                  50],
                                        'clf__min_samples_split': [2, 4, 7, 9,
                                                                   12, 14, 17,
                                                                   19, 22, 24,
                                                                   27, 29, 32,
                                                                   34, 37, 39,
                                                                   42, 44, 47,
                                                                   50]},
                   random_state=42,
                   scoring=make_scorer(fbeta_score, response_method='predict', beta=2, average=binary))

In [305]:
print(f'score F2: {rs_tree.best_score_}')
print(f'best param : {rs_tree.best_params_}')

score F2: 0.4898319634176648
best param : {'clf__min_samples_split': 27, 'clf__min_samples_leaf': 37, 'clf__max_features': None, 'clf__max_depth': 38, 'clf__criterion': 'entropy', 'clf__class_weight': None}


In [307]:
tree_tuned = rs_tree.best_estimator_
tree_tuned.fit(X_train, y_train)
y_pred  = tree_tuned.predict(X_test)
y_proba = tree_tuned.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))
print("Test F2:", rs_tree.best_score_.round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8913    0.8300    0.8595       247
           1     0.3438    0.4681    0.3964        47

    accuracy                         0.7721       294
   macro avg     0.6175    0.6490    0.6280       294
weighted avg     0.8038    0.7721    0.7855       294

Test ROC-AUC: 0.7456
Test F2: 0.4898


# Model 3 - Bagging Classifier

In [70]:
from sklearn.ensemble import BaggingClassifier

bagging_base = BaggingClassifier(random_state=42)

pipe_bagging = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTETomek(random_state=42)),
    ("clf", bagging_base)
])

In [78]:
pipe_bagging.fit(X_train, y_train)
y_pred  = pipe_bagging.predict(X_test)
y_proba = pipe_bagging.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8889    0.9393    0.9134       247
           1     0.5455    0.3830    0.4500        47

    accuracy                         0.8503       294
   macro avg     0.7172    0.6611    0.6817       294
weighted avg     0.8340    0.8503    0.8393       294

Test ROC-AUC: 0.7865


> ## HyperParameter Tuning

In [293]:
Hyperparameter_bag = {
    'clf__n_estimators': [50, 100, 200, 300, 400, 500],
    'clf__max_samples': np.linspace(0.5, 1.0, 6),      
    'clf__max_features': np.linspace(0.5, 1.0, 6),    
    'clf__bootstrap': [True, False],
    'clf__bootstrap_features': [True, False]
}

rs_bag = RandomizedSearchCV(pipe_bagging, Hyperparameter_bag, scoring=f2_scorer, random_state=42, cv=5, n_iter=50)
rs_bag.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEF787380>),
                                                                              ('cat',
                                                                               OneHotEncoder(handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEFE1FBF0>...
                                              BaggingClassifier(random_state=42))]),
                   n_iter=50,
                   param_distributions={'clf__bootstrap': [True, False],
                                        'clf__bootstrap_features': [True,
                                                                    False],
                                        'clf__max_features': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'clf__max_samples': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'clf__n_estimators': [50, 100, 200, 300,
                                                              400, 500]},
                   random_state=42,
                   scoring=make_scorer(fbeta_score, response_method='predict', beta=2, average=binary))

In [294]:
print(f'score :{rs_bag.best_score_}, best param : {rs_bag.best_params_}')

score :0.4219615957485076, best param : {'clf__n_estimators': 300, 'clf__max_samples': 0.5, 'clf__max_features': 0.7, 'clf__bootstrap_features': False, 'clf__bootstrap': True}


In [311]:
bagging_tuned = rs_bag.best_estimator_
bagging_tuned.fit(X_train, y_train)
y_pred  = bagging_tuned.predict(X_test)
y_proba = bagging_tuned.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))
print("Test F2:", rs_bag.best_score_.round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8906    0.9555    0.9219       247
           1     0.6207    0.3830    0.4737        47

    accuracy                         0.8639       294
   macro avg     0.7556    0.6692    0.6978       294
weighted avg     0.8474    0.8639    0.8502       294

Test ROC-AUC: 0.8277
Test F2: 0.422


# Model 4 - Ada Boost Classifier 

In [86]:
from sklearn.ensemble import AdaBoostClassifier

best_estimator = DecisionTreeClassifier(random_state=42)
boost_model = AdaBoostClassifier(estimator= best_estimator,
                                 algorithm='SAMME',
                                 random_state=42
                                )
pipe_boost = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTETomek(random_state=42)),
    ("clf", boost_model) 
])

In [88]:
pipe_boost.fit(X_train, y_train)
y_pred  = pipe_boost.predict(X_test)
y_proba = pipe_boost.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8689    0.8583    0.8635       247
           1     0.3000    0.3191    0.3093        47

    accuracy                         0.7721       294
   macro avg     0.5844    0.5887    0.5864       294
weighted avg     0.7779    0.7721    0.7749       294

Test ROC-AUC: 0.5887


> ## HyperParameter Tuning

In [215]:
n_estimator_boost = [200,300,400,500]
learning_rate_boost = [int(x) for x in np.linspace(1, 10, 10)]
max_depth_tree = [int(x) for x in np.linspace(20, 100, 40)]
split_tree = [int(x) for x in np.linspace(10, 100, 30)]
leaf_tree = [int(x) for x in np.linspace(10, 100, 30)]

hyperparameters = {
    'clf__n_estimators': n_estimator_boost,
    'clf__learning_rate': learning_rate_boost,
    'clf__estimator__max_depth': max_depth_tree,
    'clf__estimator__min_samples_split': split_tree,
    'clf__estimator__min_samples_leaf': leaf_tree,
    'clf__estimator__class_weight': ["balanced",None]
}

rs_boost = RandomizedSearchCV(pipe_boost, hyperparameters, scoring='average_precision', random_state=0, cv=5, n_iter=50)
rs_boost.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEF787380>),
                                                                              ('cat',
                                                                               OneHotEncoder(handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEFE1FBF0>...
                                        'clf__estimator__min_samples_leaf': [10,
                                                                             13,
                                                                             16,
                                                                             19,
                                                                             22,
                                                                             25,
                                                                             28,
                                                                             31,
                                                                             34,
                                                                             37,
                                                                             41,
                                                                             44,
                                                                             47,
                                                                             50,
                                                                             53,
                                                                             56,
                                                                             59,
                                                                             62,
                                                                             65,
                                                                             68,
                                                                             72,
                                                                             75,
                                                                             78,
                                                                             81,
                                                                             84,
                                                                             87,
                                                                             90,
                                                                             93,
                                                                             96,
                                                                             100],
                                        'clf__estimator__min_samples_split': [10,
                                                                              13,
                                                                              16,
                                                                              19,
                                                                              22,
                                                                              25,
                                                                              28,
                                                                          

In [315]:
print(f'score :{rs_boost.best_score_}, best param : {rs_boost.best_params_}')

score :0.6051851763208417, best param : {'clf__solver': 'lbfgs', 'clf__penalty': 'l2', 'clf__l1_ratio': 0.25, 'clf__class_weight': None, 'clf__C': 0.007847599703514606}


In [313]:
boost_tuned = rs_boost.best_estimator_
boost_tuned.fit(X_train, y_train)
y_pred  = boost_tuned.predict(X_test)
y_proba = boost_tuned.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))
print("Test F2:", rs_boost.best_score_.round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.9347    0.7530    0.8341       247
           1     0.3579    0.7234    0.4789        47

    accuracy                         0.7483       294
   macro avg     0.6463    0.7382    0.6565       294
weighted avg     0.8425    0.7483    0.7773       294

Test ROC-AUC: 0.7826
Test F2: 0.6052


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


# Model 4 - XGBoost Classifier 

In [155]:
from xgboost import XGBClassifier

xg_model = XGBClassifier(random_state=42)

pipe_xg = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTETomek(random_state=42)),
    ("clf", xg_model) 
])

In [157]:
pipe_xg.fit(X_train, y_train)
y_pred  = pipe_xg.predict(X_test)
y_proba = pipe_xg.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8769    0.9514    0.9126       247
           1     0.5385    0.2979    0.3836        47

    accuracy                         0.8469       294
   macro avg     0.7077    0.6246    0.6481       294
weighted avg     0.8228    0.8469    0.8280       294

Test ROC-AUC: 0.7822


> ## HyperParameter Tuning

In [203]:
n_estimator_xg = [200,400,600,800]
learning_rate_xg = [int(x) for x in np.linspace(0.01, 5, 10)]
max_depth_xg = [int(x) for x in np.linspace(2, 40, 40)]
child_weight = [int(x) for x in np.linspace(1, 20, 20)]
subsample = [int(x) for x in np.linspace(0.1, 1, 10)]
colsample = [int(x) for x in np.linspace(0.1, 1, 10)]
pos_weight = [int(x) for x in np.linspace(10, 50, 30)]

hyperparameters_xg = {
    # Jumlah pohon boosting (coba range sedang → besar)
    "clf__n_estimators": [200, 400, 600],

    # Step kontribusi tiap pohon (lebih kecil → lebih hati2)
    "clf__learning_rate": [0.01, 0.05, 0.1],

    # Kontrol kedalaman & kompleksitas pohon
    "clf__max_depth": [3, 5, 7],
    "clf__min_child_weight": [1, 3],

    # Subsampling (bantu generalisasi)
    "clf__subsample": [0.8, 1.0],
    "clf__colsample_bytree": [0.8, 1.0],

    # Imbalance handling (kalau tidak pakai SMOTE Tomek)
    "clf__scale_pos_weight": [1, 5, 10]
}

rs_xg = RandomizedSearchCV(pipe_xg, hyperparameters_xg, scoring=f2_scorer, random_state=42, cv=5, n_iter=50)
rs_xg.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEF787380>),
                                                                              ('cat',
                                                                               OneHotEncoder(handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000029FEFE1FBF0>...
                                                            num_parallel_tree=None, ...))]),
                   n_iter=50,
                   param_distributions={'clf__colsample_bytree': [0.8, 1.0],
                                        'clf__learning_rate': [0.01, 0.05, 0.1],
                                        'clf__max_depth': [3, 5, 7],
                                        'clf__min_child_weight': [1, 3],
                                        'clf__n_estimators': [200, 400, 600],
                                        'clf__scale_pos_weight': [1, 5, 10],
                                        'clf__subsample': [0.8, 1.0]},
                   random_state=42,
                   scoring=make_scorer(fbeta_score, response_method='predict', beta=2, average=binary))

In [319]:
print(f'score :{rs_xg.best_score_}, best param : {rs_xg.best_params_}')

score :0.582455929381181, best param : {'clf__subsample': 0.8, 'clf__scale_pos_weight': 10, 'clf__n_estimators': 200, 'clf__min_child_weight': 1, 'clf__max_depth': 7, 'clf__learning_rate': 0.01, 'clf__colsample_bytree': 0.8}


In [317]:
xg_best = rs_xg.best_estimator_
xg_best.fit(X_train, y_train)
y_pred  = xg_best.predict(X_test)
y_proba = xg_best.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))
print("Test F2:", rs_xg.best_score_.round(4))


=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.9316    0.7166    0.8101       247
           1     0.3269    0.7234    0.4503        47

    accuracy                         0.7177       294
   macro avg     0.6293    0.7200    0.6302       294
weighted avg     0.8349    0.7177    0.7526       294

Test ROC-AUC: 0.8042
Test F2: 0.5825


# Model 5 - Ensemble Stacking

In [6]:
import numpy as np
import pandas as pd

# split & CV
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV

# base & transformers
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# model & metrics 
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, average_precision_score, confusion_matrix


# load data
df = pd.read_csv("ibm data.csv").copy()

# map target ke 0/1
if df["Attrition"].dtype == object:
    df["Attrition"] = df["Attrition"].map({"No": 0, "Yes": 1}).astype(int)


# drop kolom
DROP_COLS = [
    "EmployeeCount","StandardHours","Over18","PerformanceRating",
    "EmployeeNumber","Education","JobLevel","PercentSalaryHike","Gender",
    "YearsAtCompany","YearsWithCurrManager","NumCompaniesWorked",
    "YearsSinceLastPromotion","RelationshipSatisfaction"
]

df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])


# feature engineering
def apply_fe(fe):
    fe = fe.copy()
    if {"YearsInCurrentRole","TotalWorkingYears"}.issubset(fe.columns):
        denom = fe["TotalWorkingYears"].replace(0, np.nan)
        fe["ExperienceRatio"] = (fe["YearsInCurrentRole"] / denom).fillna(0)

    if {"MonthlyIncome","TotalWorkingYears"}.issubset(fe.columns):
        fe["IncomePerYearExp"] = fe["MonthlyIncome"] / (fe["TotalWorkingYears"] + 1)

    if {"YearsInCurrentRole","JobSatisfaction"}.issubset(fe.columns):
        fe["TenureSatisfaction"] = fe["YearsInCurrentRole"] * fe["JobSatisfaction"]
    return fe

df_fe = apply_fe(df)

# split data
X = df_fe.drop(columns=["Attrition"])
y = df_fe["Attrition"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# encoding & scaling/standarisasi
prep = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), selector(dtype_include=np.number)),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), selector(dtype_exclude=np.number)),
    ],
    remainder="drop"
)

# pipeline
pipe = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTE(random_state=42)),
    ("clf", RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1)) # bisa tambah ata ganti model lain 
])


# ngetest doang
# cv di train 
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_f1  = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="f1")
cv_auc = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="roc_auc")
print("CV F1  :", np.round(cv_f1, 3),  "| mean =", cv_f1.mean().round(3))
print("CV AUC :", np.round(cv_auc, 3), "| mean =", cv_auc.mean().round(3))

# fit di train, predict di test
pipe.fit(X_train, y_train)
y_pred  = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]
print("\n=== TEST REPORT ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba).round(4))


CV F1  : [0.44  0.208 0.448 0.491 0.485] | mean = 0.415
CV AUC : [0.777 0.724 0.822 0.867 0.78 ] | mean = 0.794

=== TEST REPORT ===
              precision    recall  f1-score   support

           0     0.8856    0.9717    0.9266       247
           1     0.6957    0.3404    0.4571        47

    accuracy                         0.8707       294
   macro avg     0.7906    0.6560    0.6919       294
weighted avg     0.8552    0.8707    0.8516       294

Test ROC-AUC: 0.816


> ## HyperParameter Tuning

In [9]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def tune(pipe, param_dist, name, n_iter=30):
    rsearch = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_dist,
        n_iter=n_iter, scoring="f1",
        cv=cv, n_jobs=-1, verbose=1, random_state=42
    )
    rsearch.fit(X_train, y_train)
    print(f"\n[{name}] best F1: {rsearch.best_score_:.4f}")
    print(f"[{name}] best params: {rsearch.best_params_}")
    return rsearch.best_estimator_

# ========== 1) RF PIPE ==========
rf_pipe = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTE(random_state=42)),
    ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
])
rf_dist = {
    "clf__n_estimators": [300, 500, 800],
    "clf__max_depth": [None, 10, 20],
    "clf__min_samples_leaf": [1, 2, 4],
    "clf__max_features": ["sqrt", "log2"],
    "smote__k_neighbors": [3, 5]
}
rf_best = tune(rf_pipe, rf_dist, "RF")

# ========== 2) XGB PIPE ==========
xgb_pipe = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTE(random_state=42)),
    ("clf", XGBClassifier(
        eval_metric="logloss", tree_method="hist",
        random_state=42, n_jobs=-1
    ))
])
xgb_dist = {
    "clf__n_estimators": [300, 500, 800],
    "clf__learning_rate": [0.05, 0.1, 0.2],
    "clf__max_depth": [4, 6, 8],
    "clf__subsample": [0.8, 1.0],
    "clf__colsample_bytree": [0.8, 1.0],
    "smote__k_neighbors": [3, 5]
}
xgb_best = tune(xgb_pipe, xgb_dist, "XGB")

# ========== 3) LR PIPE ==========
lr_pipe = ImbPipeline(steps=[
    ("prep", prep),  # scaling penting buat LR
    ("smote", SMOTE(random_state=42)),
    ("clf", LogisticRegression(max_iter=2000, solver="lbfgs"))
])
lr_dist = {
    "clf__C": [0.1, 0.5, 1.0, 2.0, 5.0],
    "clf__class_weight": [None, "balanced"],
    "smote__k_neighbors": [3, 5]
}
lr_best = tune(lr_pipe, lr_dist, "LR")

# ========== 4) EVAL CEPAT per model ==========
def quick_test(name, best_pipe):
    best_pipe.fit(X_train, y_train)
    proba = best_pipe.predict_proba(X_test)[:,1]
    y_pred = (proba >= 0.5).astype(int)  # optional: nanti bisa threshold tuning
    print(f"\n=== {name} (thr=0.5) ===")
    print(classification_report(y_test, y_pred, digits=4))
    print("ROC-AUC:", roc_auc_score(y_test, proba).round(4))

quick_test("RF", rf_best)
quick_test("XGB", xgb_best)
quick_test("LR",  lr_best)

Fitting 5 folds for each of 30 candidates, totalling 150 fits



[RF] best F1: 0.4696
[RF] best params: {'smote__k_neighbors': 5, 'clf__n_estimators': 500, 'clf__min_samples_leaf': 4, 'clf__max_features': 'sqrt', 'clf__max_depth': None}
Fitting 5 folds for each of 30 candidates, totalling 150 fits

[XGB] best F1: 0.5003
[XGB] best params: {'smote__k_neighbors': 5, 'clf__subsample': 1.0, 'clf__n_estimators': 300, 'clf__max_depth': 6, 'clf__learning_rate': 0.1, 'clf__colsample_bytree': 1.0}
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



[LR] best F1: 0.4940
[LR] best params: {'smote__k_neighbors': 5, 'clf__class_weight': None, 'clf__C': 5.0}

=== RF (thr=0.5) ===
              precision    recall  f1-score   support

           0     0.8902    0.9514    0.9198       247
           1     0.6000    0.3830    0.4675        47

    accuracy                         0.8605       294
   macro avg     0.7451    0.6672    0.6936       294
weighted avg     0.8438    0.8605    0.8475       294

ROC-AUC: 0.8174

=== XGB (thr=0.5) ===
              precision    recall  f1-score   support

           0     0.8778    0.9595    0.9168       247
           1     0.5833    0.2979    0.3944        47

    accuracy                         0.8537       294
   macro avg     0.7306    0.6287    0.6556       294
weighted avg     0.8307    0.8537    0.8333       294

ROC-AUC: 0.7795

=== LR (thr=0.5) ===
              precision    recall  f1-score   support

           0     0.9303    0.7571    0.8348       247
           1     0.3548    0.7

In [10]:
# ambil model inti (tanpa prep/smote) dari best pipes
rf_clf  = rf_best.named_steps["clf"]
xgb_clf = xgb_best.named_steps["clf"]
lr_clf  = lr_best.named_steps["clf"]

# definisi stacking (meta-learner = LR)
stack_clf = StackingClassifier(
    estimators=[("rf", rf_clf), ("xgb", xgb_clf), ("lr", lr_clf)],
    final_estimator=LogisticRegression(max_iter=2000),
    stack_method="predict_proba",
    passthrough=True,
    n_jobs=-1
)

# pipeline stacking lengkap (prep → SMOTE → stacking)
stack_pipe = ImbPipeline(steps=[
    ("prep", prep),
    ("smote", SMOTE(random_state=42)),
    ("clf", stack_clf),
])

# --- evaluasi CV di TRAIN (cepat) ---
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_f1  = cross_val_score(stack_pipe, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
cv_auc = cross_val_score(stack_pipe, X_train, y_train, cv=cv, scoring="roc_auc", n_jobs=-1)
print("STACK (no tuning) | CV F1 :", np.round(cv_f1, 3),  "| mean =", cv_f1.mean().round(3))
print("STACK (no tuning) | CV AUC:", np.round(cv_auc, 3), "| mean =", cv_auc.mean().round(3))

# --- fit & test ---
stack_pipe.fit(X_train, y_train)
proba_test = stack_pipe.predict_proba(X_test)[:, 1]
y_pred     = (proba_test >= 0.5).astype(int)   # threshold default 0.5
print("\n=== STACK (no tuning) — TEST ===")
print(classification_report(y_test, y_pred, digits=4))print("Test ROC-AUC:", roc_auc_score(y_test, proba_test).round(4))

STACK (no tuning) | CV F1 : [0.542 0.407 0.585 0.618 0.521] | mean = 0.534
STACK (no tuning) | CV AUC: [0.825 0.76  0.841 0.878 0.804] | mean = 0.821

=== STACK (no tuning) — TEST ===
              precision    recall  f1-score   support

           0     0.8859    0.9433    0.9137       247
           1     0.5484    0.3617    0.4359        47

    accuracy                         0.8503       294
   macro avg     0.7172    0.6525    0.6748       294
weighted avg     0.8320    0.8503    0.8373       294

Test ROC-AUC: 0.7922


In [11]:
# param dist untuk tuning ringan 
stack_dist = {
    # meta-learner (LogReg)
    "clf__final_estimator__C": [0.5, 1.0, 2.0, 5.0],
    # base RF
    "clf__rf__n_estimators": [300, 500, 800],
    "clf__rf__max_depth": [None, 10, 20],
    "clf__rf__min_samples_leaf": [1, 2, 4],
    "clf__rf__max_features": ["sqrt", "log2"],
    # base XGB
    "clf__xgb__n_estimators": [300, 500, 800],
    "clf__xgb__learning_rate": [0.05, 0.1, 0.2],
    "clf__xgb__max_depth": [4, 6, 8],
    "clf__xgb__subsample": [0.8, 1.0],
    "clf__xgb__colsample_bytree": [0.8, 1.0],
    # SMOTE
    "smote__k_neighbors": [3, 5]
}

rsearch_stack = RandomizedSearchCV(
    estimator=stack_pipe,
    param_distributions=stack_dist,
    n_iter=40,                    # naikin kalau mau explore lebih luas
    scoring="f1",                 # bisa ganti ke F2 kalau fokus recall
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

rsearch_stack.fit(X_train, y_train)
print("\n[STACK] Best params:", rsearch_stack.best_params_)
print("[STACK] Best CV F1 :", round(rsearch_stack.best_score_, 4))

stack_best = rsearch_stack.best_estimator_

# --- evaluasi di test dengan threshold default 0.5 ---
stack_best.fit(X_train, y_train)
proba_test = stack_best.predict_proba(X_test)[:, 1]
y_pred     = (proba_test >= 0.5).astype(int)
print("\n=== STACK (tuned; thr=0.5) — TEST ===")
print(classification_report(y_test, y_pred, digits=4))
print("Test ROC-AUC:", roc_auc_score(y_test, proba_test).round(4))


Fitting 5 folds for each of 40 candidates, totalling 200 fits

[STACK] Best params: {'smote__k_neighbors': 5, 'clf__xgb__subsample': 0.8, 'clf__xgb__n_estimators': 300, 'clf__xgb__max_depth': 6, 'clf__xgb__learning_rate': 0.05, 'clf__xgb__colsample_bytree': 1.0, 'clf__rf__n_estimators': 300, 'clf__rf__min_samples_leaf': 2, 'clf__rf__max_features': 'sqrt', 'clf__rf__max_depth': None, 'clf__final_estimator__C': 0.5}
[STACK] Best CV F1 : 0.5362

=== STACK (tuned; thr=0.5) — TEST ===
              precision    recall  f1-score   support

           0     0.8893    0.9433    0.9155       247
           1     0.5625    0.3830    0.4557        47

    accuracy                         0.8537       294
   macro avg     0.7259    0.6631    0.6856       294
weighted avg     0.8371    0.8537    0.8420       294

Test ROC-AUC: 0.7974
